# Install Libraries

In [3]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install pulp

In [13]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import gym
from gym import spaces
import numpy as np

# Data Preparation

In [7]:
# load the task data
task_data = pd.read_excel(r"C:\Users\slayj\OneDrive\Documents\dql-may\dql_Task_Offloading\archive\task40.xlsx", sheet_name='TaskDetails')

# load the node data
node_data = pd.read_excel(r"C:\Users\slayj\OneDrive\Documents\dql-may\dql_Task_Offloading\archive\task40.xlsx", sheet_name='NodeDetails')

# Remove "Unnamed: 0" column
task_data = task_data.drop(columns=["Unnamed: 0"])
node_data = node_data.drop(columns=["Unnamed: 0"])



In [8]:
from sklearn.model_selection import train_test_split

# split the task data into a training set and a test set
task_train, task_test = train_test_split(task_data, test_size=0.2)

# split the node data into a training set and a test set
node_train, node_test = train_test_split(node_data, test_size=0.2)


In [9]:
node_data

,CPU rate (MIPS),CPU usage cost,Memory usage cost,Bandwidth usage cost
0,3754,0.8044,0.02088,0.0571
1,3585,0.9433,0.03190,0.0806
2,4617,0.8208,0.04455,0.0563
3,1097,0.2464,0.02723,0.0105
4,873,0.1360,0.01474,0.0180
5,1054,0.1142,0.01232,0.0183
6,1420,0.3196,0.01802,0.0113
7,1106,0.3844,0.01195,0.0181
8,1479,0.1576,0.02440,0.0129
9,645,0.2805,0.01828,0.0115


In [10]:
task_data

,Number of instructions (109 instructions),Memory required (MB),Input file size (MB),Output file size (MB)
0,40,114,71,28
1,21,57,44,25
2,45,81,64,10
3,71,90,78,99
4,92,58,60,20
5,36,107,90,31
6,21,198,38,56
7,72,52,99,43
8,33,121,81,70
9,14,145,42,82


# Create the Environment

In [11]:
class TaskAssignmentEnvironment(gym.Env):
    def __init__(self, task_df, node_df):
        super(TaskAssignmentEnvironment, self).__init__()

        self.task_df = task_df
        self.node_df = node_df

        self.num_tasks = len(self.task_df)
        self.num_nodes = len(self.node_df)

        self.action_space = spaces.MultiDiscrete([self.num_nodes] * self.num_tasks)

        low_values = np.concatenate((np.zeros(self.num_tasks * 4), np.zeros(self.num_nodes * 4)))
        high_values = np.concatenate((np.ones(self.num_tasks * 4) * np.inf, np.ones(self.num_nodes * 4) * np.inf))
        self.observation_space = spaces.Box(low=low_values, high=high_values)

    def step(self, action):
        total_cost = 0

        for task_idx, node_idx in enumerate(action):
            task = self.task_df.iloc[task_idx]
            node = self.node_df.iloc[node_idx]

            exec_time = task["Number of instructions (109 instructions)"] / node["CPU rate (MIPS)"] if node["CPU rate (MIPS)"] != 0 else np.inf

            task_cost = exec_time * node["CPU usage cost"] + task["Memory required (MB)"] * node["Memory usage cost"] + task["Input file size (MB)"] * node["Bandwidth usage cost"]
            total_cost += task_cost

        reward = -total_cost

        done = True
        info = {}

        return self._get_state(), reward, done, info

    def reset(self):
        return self._get_state()

    def render(self, mode='human'):
        pass

    def _get_state(self):
        return np.concatenate((self.task_df.values.flatten(), self.node_df.values.flatten()))


In [14]:
# Create the environment
env = TaskAssignmentEnvironment(task_data, node_data)